In [105]:
import pandas as pd
import numpy as np
import scipy.interpolate as si

In [106]:
synth_pop_original = pd.read_feather("synth_pop_original.feather")
print(synth_pop_original)

          age    size ownership           family_comp
0       60_74  2_pers     Owner  Couple_without_child
1       60_74  2_pers     Owner  Couple_without_child
2       60_74  2_pers     Owner  Couple_without_child
3        0_29  1_pers    Tenant            Single_wom
4       60_74  2_pers     Owner  Couple_without_child
...       ...     ...       ...                   ...
157642   0_29  1_pers    Tenant            Single_man
157643   0_29  1_pers    Tenant            Single_wom
157644   0_29  1_pers    Tenant            Single_wom
157645   0_29  1_pers    Tenant            Single_wom
157646   0_29  1_pers    Tenant            Single_wom

[157647 rows x 4 columns]


In [107]:
deciles_filosofi = pd.read_feather("deciles_filosofi.feather")
print(deciles_filosofi)

                    modality            D1            D2            D3  \
0                   1 person   9794.000000  12961.000000  14914.000000   
1                  2 persons  12176.000000  15552.666667  18356.000000   
2                  3 persons  10583.500000  13655.555556  16489.000000   
3                  4 persons  10740.476190  14130.000000  17206.666667   
4          5 persons or more   8758.461538  10990.000000  12878.666667   
5                 Single man   9016.000000  12223.500000  14288.000000   
6               Single woman  10714.000000  13334.000000  15332.000000   
7   Couple  without children  14417.333333  18066.000000  20791.333333   
8      Couple  with children  10822.380952  14237.826087  17645.714286   
9       Single parent family   8702.500000  10367.200000  11914.666667   
10        Complex households   8692.000000  11052.121212  13063.076923   
11                      0_29   8370.625000  11116.666667  13501.000000   
12                     30_39   9985.38

In [108]:
# dec_owner = synth_pop_original["ownership"].unique()   # ["Owner", "Tenant"]
# dec_fam_comp = synth_pop_original["family_comp"].unique()  # ["Single_man", "Single_wom", "Couple_without_child", "Couple_with_child", "Single_parent", "complex_hh"]
# dec_age = synth_pop_original["age"].unique()  # ["0_29", "30_39",  "40_49", "50_59", "60_74", "75_or_more"]
# dec_size = synth_pop_original["size"].unique()  # ["1_pers", "2_pers", "3_pers", "4_pers", "5_pers_or_more"]

In [109]:
# dec_owner = ["Owner", "Tenant"]
# dec_fam_comp = ["Single_man", "Single_wom", "Couple_without_child", "Couple_with_child", "Single_parent", "complex_hh"]
# dec_age = ["0_29", "30_39",  "40_49", "50_59", "60_74", "75_or_more"]
# dec_size = ["1_pers", "2_pers", "3_pers", "4_pers", "5_pers_or_more"]

In [110]:
# tmp_fam_comp, tmp_owner, tmp_age, tmp_size = np.meshgrid(dec_fam_comp, dec_owner, dec_age, dec_size)
# synt_pop_comb = pd.DataFrame.from_dict({
#     "ownership": tmp_owner.flatten(),
#     "age": tmp_age.flatten(),
#     "size": tmp_size.flatten(),
#     "family_comp": tmp_fam_comp.flatten()
# })
# synt_pop_comb["total"] = synt_pop_comb[["ownership", "age" ,"size", "family_comp"]].agg(' '.join, axis=1)

# print(synt_pop_comb)

In [111]:
synth_pop_original["count"] = 1
synt_pop_comb = synth_pop_original.groupby(["family_comp", "size" ,"age", "ownership"])["count"].sum().fillna(0.0).reset_index()
synt_pop_comb["total"] = synt_pop_comb[["ownership", "age" ,"size", "family_comp"]].agg(' '.join, axis=1)
synt_pop_comb["probability"] = synt_pop_comb["count"] / synt_pop_comb["count"].sum()
#reorder col
synt_pop_comb = synt_pop_comb[["ownership", "age" ,"size", "family_comp", "total", "count", "probability"]]

print(synt_pop_comb)

    ownership         age            size family_comp  \
0      Tenant        0_29          1_pers  Single_wom   
1       Owner        0_29          1_pers  Single_wom   
2      Tenant       30_39          1_pers  Single_wom   
3       Owner       30_39          1_pers  Single_wom   
4      Tenant       40_49          1_pers  Single_wom   
..        ...         ...             ...         ...   
355     Owner       50_59  5_pers_or_more  complex_hh   
356    Tenant       60_74  5_pers_or_more  complex_hh   
357     Owner       60_74  5_pers_or_more  complex_hh   
358    Tenant  75_or_more  5_pers_or_more  complex_hh   
359     Owner  75_or_more  5_pers_or_more  complex_hh   

                                           total    count  probability  
0                  Tenant 0_29 1_pers Single_wom  14613.0     0.092694  
1                   Owner 0_29 1_pers Single_wom    485.0     0.003076  
2                 Tenant 30_39 1_pers Single_wom   3308.0     0.020984  
3                  Owne

In [113]:
deciles_filosofi["D10"] = 1.5 * deciles_filosofi["D9"]
deciles_filosofi["D0"] = 0

vec_all_incomes = deciles_filosofi[["D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10"]].to_numpy().flatten()
vec_all_incomes = np.sort(vec_all_incomes)

total_population_decile = [0.00, 10303.48, 13336.07, 16023.85, 18631.33, 21262.67, 24188.00, 27774.44, 32620.00, 41308.00, 75090.00]
f = si.interp1d(total_population_decile, np.linspace(0, 1, len(total_population_decile)))
p_R = pd.DataFrame({
    "income": vec_all_incomes,
    "probability": f(vec_all_incomes)
})

print(p_R)

           income  probability
0     8370.625000     0.081241
1     8692.000000     0.084360
2     8702.500000     0.084462
3     8758.461538     0.085005
4     8764.444444     0.085063
..            ...          ...
185  67465.500000     0.977430
186  69987.000000     0.984894
187  70910.000000     0.987627
188  72822.000000     0.993286
189  75090.000000     1.000000

[190 rows x 2 columns]


In [ ]:
# try with https://github.com/PythonCharmers/maxentropy
# we want KL divergence
# priorlogprobs = P('homme' & 'proptio')
# Sortie=minxent.multiple(q=final_joint_proba,G=Moment_mat[Iii,],eta = eta_c[Iii,i],lambda = rep(0,length(Iii)-1)) # mieux gérer les lambda
# q = priorlogprobs
# G = samplesize
# eta = K
# lambda = (tout à 0?) (facteurs lagrange initiaux)